<a href="https://colab.research.google.com/github/nedelculiviu95/cnn-detecting-lung-opacities/blob/master/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install pydicom
import glob, pylab, pandas as pd
import pydicom, numpy as np

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def parse_data(df):
    """
      parsed = {
        
        'image-name01': {
            'png': path/to/png/file,
            'label': list of labels, 
            'patient_gender': M or F
            'pattient_age': int
        },
        'image-name-02': {
            'png': path/to/png/file,
            'label': list of labels,
            'patient_gender': M or F
            'pattient_age': int
        }, ...

      }
      
    """
    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['Image Index']
        if pid not in parsed:
            labels = row['Finding Labels']
            parsed[pid] = {
                'png': '%s' % pid,
                'label': labels.split("|"),
                'patient_gender': row['Patient Gender'],
                'pattient_age': row['Patient Age']}
        
    return parsed
  
  
df = pd.read_csv('/content/gdrive/My Drive/ChestXRay_csv/Data_Entry_2017.csv')
parsed = parse_data(df)



In [0]:
folder3 = '/content/gdrive/My Drive/validation_labels.txt'
folder2 = '/content/gdrive/My Drive/test_labels.txt'
folder = '/content/gdrive/My Drive/train_filenmaes.txt'
validation = []
f=open(folder3,'r')
for line in f.readlines():
    validation.append(line)

validation = validation[9:]
validation_filenames = []
for p in validation:
    nn = p[0:16]
    validation_filenames.append(nn)

print("Validation data: ", len(validation_filenames))
   
train = []
f=open(folder,'r')
for line in f.readlines():
    train.append(line)
    
train = train[9:]
train_filenames = []
for p in train:
    nn = p[0:16]
    train_filenames.append(nn)
print("Training data: ",len(train_filenames))
   
test = []
f=open(folder2,'r')
for line in f.readlines():
    test.append(line)
test = test[9:]
test_filenames = []
for p in test:
    nn = p[0:16]
    test_filenames.append(nn)

print("Validation data: ",len(test_filenames))

Validation data:  11219
Training data:  78468
Validation data:  22433


## Ratio for the entire dataset

In [0]:
train_data = len(train_filenames)
validation_data = len(validation_filenames)
test_data = len(test_filenames)


def diseases_breakdown(parsed, filenames):
  dictionary = {'Atelectasis': 0, 'Cardiomegaly': 0, 'Effusion': 0, 'Infiltration':0, 
                'Mass': 0, 'Nodule': 0, 'Pneumonia': 0, 'Pneumothorax': 0, 'Consolidation': 0,
               'Edema':0, 'Emphysema':0, 'Fibrosis': 0, 'Pleural_Thickening': 0, 'Hernia': 0, 'No Finding': 0}
  for fil in filenames:
    labels = parsed[fil]['label']
    
    for l in labels:
      dictionary[l] += 1
  
  return dictionary

def disease_table(dictionary, filenames, dataset_type):
  total = 0
  ratio_list = []
  print("Total",dataset_type,": ",len(filenames))
  print("---------------------------------")
  for key,value in dictionary.items():
    ratio = value / len(filenames)
    ratio_list.append(ratio)
    print(key,":",value, "      Ratio: ", ratio)
  print()
  return ratio_list

In [0]:
dd = diseases_breakdown(parsed, validation_filenames)
validation_ratio_list = disease_table(dd, validation_filenames, "validation data")
dd = diseases_breakdown(parsed, train_filenames)
train_ratio_list = disease_table(dd, train_filenames, "train data")
dd = diseases_breakdown(parsed, test_filenames)
test_ratio_list = disease_table(dd, test_filenames, "test data")

## Ratio for the experiments dataset

In [0]:
import os
from os.path import isfile, join

images1_folder = "/content/gdrive/My Drive/images1/"
images3_folder = "/content/gdrive/My Drive/images/"
filenames_3 = [f for f in os.listdir(images3_folder) if isfile(join(images3_folder, f))]
filenames_1 = [f for f in os.listdir(images1_folder) if isfile(join(images1_folder, f))]
#filenames = filenames_1 + filenames_3
#print(len(filenames))
filenames_3.sort()
filenames_1.sort()


n_valid_samples = 1400
n_train_examples = 10000
n_test_examples = 2900

sample_train_filenames = filenames_3
sample_valid_filenames = filenames_1[0:n_valid_samples]
sample_test_filenames = filenames_1[n_valid_samples:n_valid_samples+n_test_examples:]
print('n sample train samples', len(sample_train_filenames))
print('n sample valid samples', len(sample_valid_filenames))
print('n sample test samples', len(sample_test_filenames))



## Validation set new distribution

In [0]:
def redistribute(source_dataset, target_dataset, parsed, new_dataset, filenames_used, filenames_size):
  f_used = []
  labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration','Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation','Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia','No Finding'] 
  target_label_increase = [2, 4]
  size = filenames_size
  
  for filename in new_dataset:
    l = parsed[filename]['label']
    if filename not in filenames_used and size > 1:
        if 'Effusion' in l or 'Mass' in l:
          source_dataset.append(filename)
          f_used.append(filename)
          size -= 1
  
  dd1 = diseases_breakdown(parsed, source_dataset)
  disease_table(dd1, source_dataset, "New data distribution")
  return f_used

In [0]:
dd = diseases_breakdown(parsed, sample_valid_filenames)
sample_validation_ratio_list = disease_table(dd, sample_valid_filenames, "validation data")
dd = diseases_breakdown(parsed, sample_train_filenames)
sample_training_ratio_list = disease_table(dd, sample_train_filenames, "train data")
dd = diseases_breakdown(parsed, sample_test_filenames)
sample_test_ratio_list = disease_table(dd, sample_test_filenames, "test data")

## Disease distribution for the new dataset

In [0]:
import os
from os.path import isfile, join

images2_folder = "/content/gdrive/My Drive/images1/"
filenames_2 = [f for f in os.listdir(images2_folder) if isfile(join(images2_folder, f))]
#filenames = filenames_1 + filenames_3
#print(len(filenames))

filenames_2.sort()

print('n samples', len(filenames_2))

dd = diseases_breakdown(parsed, filenames_2)
disease_table(dd, filenames_2, "new data")

## Redistribute data to match ratio from the entire dataset distribution

In [0]:
total_f_used = filenames_used_for_trainig + f_used_for_testing

add_nf = 100
aux_validation_f = filenames_1[0:n_valid_samples]
for f in filenames_2:
  if f not in total_f_used and parsed[f]['label'][0] == 'No Finding':
    aux_validation_f.append(f)
    add_nf -= 1
    if add_nf == 0:
      break

dd2 = diseases_breakdown(parsed, aux_validation_f)
disease_table(dd2, aux_validation_f, "New data distribution")

### Training

In [0]:
def redistribute(source_dataset, parsed, new_dataset):
  filenames_used_for_trainig = []
  labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration','Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation','Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia','No Finding'] 
  target_label_increase = [0, 2, 3, 4, 7, 8, 9, 10]
        
  for filename in new_dataset:
    l = parsed[filename]['label']
    perfect_match = 0
    for dd in l:
      for k in range(len(labels)):
        if dd == labels[k] and k in target_label_increase:
          perfect_match += 1
      if perfect_match == len(l):
        source_dataset.append(filename)
        filenames_used_for_trainig.append(filename)
      elif 'Pneumonia' in l and len(l) < 3:
        source_dataset.append(filename)
        filenames_used_for_trainig.append(filename)
  
  d_take = 130
  d_take1 = 100
  for ff in source_dataset:
    if 'Infiltration' in parsed[ff]['label'] and len(parsed[ff]['label']) > 1:
      source_dataset.remove(ff)
      d_take -= 1
      if d_take == 0:
        break 
    elif 'Atelectasis' in parsed[ff]['label'] and len(parsed[ff]['label']) > 1:
      source_dataset.remove(ff)
      d_take1 -= 1
      if d_take1 == 0:
        break
 
  dd = diseases_breakdown(parsed, source_dataset)
  disease_table(dd, source_dataset, "New data distribution")
  return filenames_used_for_trainig
##### new data distribution for training sample 
print(len(sample_train_filenames))
s_t_f = sample_train_filenames
filenames_used_for_trainig = redistribute(s_t_f, parsed, filenames_2)

### Testing

In [0]:
remove_nf = 320
aux_test_f = filenames_1[n_valid_samples:n_valid_samples+n_test_examples:]
for i in range(len(aux_test_f)-1, 0, -1):
  ff = aux_test_f[i]
  if parsed[ff]['label'][0] == 'No Finding':
    aux_test_f.remove(ff)
    remove_nf -= 1
    if remove_nf == 0:
      break

def redistribute(source_dataset, target_dataset, parsed, new_dataset, filenames_used, filenames_size):
  f_used = []
  labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration','Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation','Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia','No Finding'] 
  target_label_increase = [2, 4]
  size = filenames_size
  
  for filename in new_dataset:
    l = parsed[filename]['label']
    if filename not in filenames_used and size > 1:
        if 'Effusion' in l or 'Mass' in l:
          source_dataset.append(filename)
          f_used.append(filename)
          size -= 1
  
  dd1 = diseases_breakdown(parsed, source_dataset)
  disease_table(dd1, source_dataset, "New data distribution")
  return f_used

print(len(sample_test_filenames)) 
f_used_for_testing = redistribute(aux_test_f, test_filenames, parsed, filenames_2, filenames_used_for_trainig, 150)

## Breakdown image filenames by diseases

In [0]:
##### new data distribution for training sample 
print(len(sample_train_filenames))
s_t_f = sample_train_filenames
redistribute(s_t_f, sample_training_ratio_list, train_filenames, train_ratio_list, parsed, filenames_2)

In [0]:
def disease_distribution(parsed):
  dictionary = {'Lung Opacity': 0, 'Normal': 0, 'No Lung Opacity / Not Normal': 0}
  for key, value in parsed.items():
    labels = parsed[key]['label']
    for item in (labels):
      dictionary[item] += 1
      
  return dictionary


disease_count = disease_distribution(parsed)
print(disease_count)

plt.figure(figsize=(20,10))
plt.bar(range(len(disease_count)), list(disease_count.values()), width=0.8, align='center')
plt.xticks(range(len(disease_count)), list(disease_count.keys()))

plt.show()